In [1]:
%pip install langchain langchain_openai PyPDF2 faiss-cpu chromadb google-api-core google-api-python-client google-auth google-auth-httplib2 googleapis-common-protos html2text --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.retrievers.web_research import WebResearchRetriever
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from dotenv import load_dotenv

EMBEDDINGS_DEPLOYMENT_NAME = "text-embedding-ada-002"
EMBEDDINGS_API_VERSION = "2023-08-01-preview"

DEPLOYMENT_NAME="gpt-35-turbo"
MODEL_NAME = "gpt-35-turbo"
API_VERSION="2023-12-01-preview"

load_dotenv()

True

In [2]:
# Vectorstore
vectorstore = Chroma(
    embedding_function=AzureOpenAIEmbeddings(
        azure_deployment=EMBEDDINGS_DEPLOYMENT_NAME,
        openai_api_version=EMBEDDINGS_API_VERSION), 
        persist_directory="./chroma_db_oai"
)

# LLM
llm = AzureChatOpenAI(
    azure_deployment=DEPLOYMENT_NAME, 
    model_name=MODEL_NAME, 
    openai_api_version=API_VERSION,
    temperature=0)

# Search
search = GoogleSearchAPIWrapper()

In [4]:
# Initialize
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore, llm=llm, search=search
)

In [7]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)
from langchain.chains import RetrievalQAWithSourcesChain

user_input = "How do LLM Powered Autonomous Agents work?"
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, retriever=web_research_retriever, return_source_documents=True
)

qa_chain.invoke({"question": user_input})


INFO:langchain.retrievers.web_research:Generating questions for Google Search ...


INFO:langchain.retrievers.web_research:Questions for Google Search (raw): {'question': 'How do LLM Powered Autonomous Agents work?', 'text': LineList(lines=['1. What is the functioning principle of LLM Powered Autonomous Agents?\n', '2. How do LLM Powered Autonomous Agents operate?\n', '3. Can you explain the working mechanism of LLM Powered Autonomous Agents?'])}
INFO:langchain.retrievers.web_research:Questions for Google Search: ['1. What is the functioning principle of LLM Powered Autonomous Agents?\n', '2. How do LLM Powered Autonomous Agents operate?\n', '3. Can you explain the working mechanism of LLM Powered Autonomous Agents?']
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Search results: [{'title': "LLM Powered Autonomous Agents | Lil'Log", 'link': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'snippet': 'Jun 23, 2023 ... 1. Overview of

{'question': 'How do LLM Powered Autonomous Agents work?',
 'answer': "LLM-powered autonomous agents work by utilizing LLM as the agent's brain, along with several key components such as planning, memory, and tool use. In terms of planning, the agent breaks down large tasks into smaller subgoals and can reflect on past actions to improve future results. The agent also utilizes short-term and long-term memory to learn and retain information. Additionally, the agent can call external APIs for additional information. The design of generative agents combines LLM with memory, planning, and reflection mechanisms to enable agents to behave based on past experience and interact with other agents.\n",
 'sources': '',
 'source_documents': [Document(page_content="# Agent System Overview#\n\nIn a LLM-powered autonomous agent system, LLM functions as the agent's brain,\ncomplemented by several key components:\n\n  * **Planning**\n    * Subgoal and decomposition: The agent breaks down large tasks in